<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [12]:
from imports import *

<h4>preprocessing and feature extraction</h4>

In [14]:
one_hot_enconding = np.eye(37) 
def Tokenizing(dataset_sentences):
    harakat  = {
                    "": 0,     # No Diacritic
                    "َ": 1,     # Fatha
                    "ً": 2,     # Fathatah
                    "ُ": 3,     # Damma
                    "ٌ": 4,     # Dammatan
                    "ِ": 5,     # Kasra
                    "ٍ": 6,     # Kasratan
                    "ْ": 7,     # Sukun
                    "ّ": 8,     # Shaddah
                    "َّ": 9,     # Shaddah + Fatha
                    "ًّ": 10,    # Shaddah + Fathatah
                    "ُّ": 11,    # Shaddah + Damma
                    "ٌّ": 12,    # Shaddah + Dammatan
                    "ِّ": 13,    # Shaddah + Kasra
                    "ٍّ": 14,    # Shaddah + Kasratan
                            
                }
        
    arabic_alphabet = {
                            "ا": 1,
                            "ب": 2,
                            "ت": 3,
                            "ث": 4,
                            "ج": 5,
                            "ح": 6,
                            "خ": 7,
                            "د": 8,
                            "ذ": 9,
                            "ر": 10,
                            "ز": 11,
                            "س": 12,
                            "ش": 13,
                            "ص": 14,
                            "ض": 15,
                            "ط": 16,
                            "ظ": 17,
                            "ع": 18,
                            "غ": 19,
                            "ف": 20,
                            "ق": 21,
                            "ك": 22,
                            "ل": 23,
                            "م": 24,
                            "ن": 25,
                            "ه": 26,
                            "و": 27,
                            "ي": 28,
                            "آ": 29,
                            "إ": 30,
                            "ئ": 31,
                            "ء": 32,
                            "أ": 33,
                            "ؤ":34,
                            "ة":35,
                            "ى":36,
                        }

    x_train_letters = []
    y_train_letters = []
    x_train_words = []
    for sentence in dataset_sentences:
        sentence_wihout_diacratics = strip_tashkeel(sentence)
        tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
        tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)

        sentence_x = list()
        sentence_y = list()
        for word in tokens:
            text, inputs, diacritics =util.extract_haraqat(word)
            
            word_x = list()
            word_y = list()
            for i in range(len(inputs)):
                # for every char we see its rep in arabic_alphabet and get its 1 hot encoding vector
                word_x.append(arabic_alphabet[inputs[i]])
                word_y.append(harakat[diacritics[i]])
                # word_x.append(inputs[i])
                # word_y.append(diacritics[i])
            sentence_x.append(word_x)
            sentence_y.append(word_y)
        if(len(sentence_x) != 0):
            # 4d array len of sentences * len of words * len of chars * one hot encoding size
            x_train_letters.append(sentence_x)
            # 3d array len of sentences * len of words * number of diacritics --> represents the diacritics as indices check (harakat dict) 
            y_train_letters.append(sentence_y)

        if(len(tokens_wihtout_diacratics)) :
            x_train_words.append(tokens_wihtout_diacratics)


    print(y_train_letters[1])
    print(len(x_train_letters), len(x_train_letters[1]), len(x_train_letters[1][1]), x_train_letters[1][1][1])
    print(x_train_words[1])
    return x_train_letters,y_train_letters,x_train_words

dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
chars_to_remove = r"\(\s*[a-zA-Z0-9_-`]+\s*/\s*[a-zA-Z0-9_-`]+\s*\)|[a-zA-Z0-9_-]+|-|`|–|~|\u200f|'" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*|\s*\[\s*|\s*\]\s*|\s*{\s*|\s*}\s*|\s*\*\s*|\s*»\s*|\s*«\s*|\s*\\\s*|\s*/\s*|\s*;\s*|\s*,\s*",dataset_cleaned)

x_train_chars,y_train_chars,x_train_words = Tokenizing(dataset_sentences)

[[1, 7], [1, 1, 1], [0, 7, 1, 9, 3], [1, 1, 3], [0, 1, 7]]
278416 5 3 16
['أو', 'قطع', 'الأول', 'يده', 'إلخ']


In [15]:
model = Word2Vec(sentences=x_train_words, vector_size=100, window=5, min_count=1, workers=4)

model.save("my_word2vec_model")

In [16]:
word2vec_model = Word2Vec.load("my_word2vec_model")
embedding_matrix = np.zeros((len(word2vec_model.wv.index_to_key) + 1, word2vec_model.vector_size))

word_index_x = {word: idx + 1 for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, idx in word_index_x.items():
    embedding_matrix[idx] = word2vec_model.wv[word]

x_train_words_indices = [[word_index_x[word] for word in sentence] for sentence in x_train_words]

<h3>Dataset class<h3>

In [21]:
class ArabicDataset(Dataset):

    def __init__(self, x_train_words, x_train_chars, y_train_chars, one_hot_enconding):
        # the words dataset that will be given to the first model
        max_list_length = max(len(_) for _ in x_train_words)
        x_padded = [_ + [0] * (max_list_length - len(_)) for _ in x_train_words]
        self.tensor_x_words = torch.tensor(x_padded)

        x_padded_chars = [_ + [[]] * (max_list_length - len(_)) for _ in x_train_chars]
        #print(x_padded_chars[1])
        #print(len(x_padded_chars[1]),len(x_padded_chars[1][1]))

        # for sentence in x_train_chars:
        #     temp=list()
        #     for i in range(max_list_length - len(sentence)):
        #         temp.append([])
        #     sentence.extend(temp)

        max_char_length = max(len(word) for sentence in x_train_chars for word in sentence)
        print(len(x_padded_chars), len(x_padded_chars[1]), len(x_padded_chars[1][1]), x_padded_chars[1][1][1])
        print(len(x_padded_chars), len(x_padded_chars[2]), len(x_padded_chars[2][1]), x_padded_chars[2][1][1])
        for i in range(len(x_padded_chars)): #0->253
            for j in range(len(x_padded_chars[i])):
                temp=list()
                for k in range(max_char_length - len(x_padded_chars[i][j])):
                    temp.append(0)
                #print(temp)
                if(len(temp)!=0):
                    x_padded_chars[i][j].extend(temp)
        
        print(len(x_padded_chars[0]))
        print(len(x_padded_chars), len(x_padded_chars[2]), len(x_padded_chars[2][1]), x_padded_chars[2][1][1])
        self.tensor_x_chars = torch.tensor(x_padded_chars)

        max_char_length = max(len(word) for sentence in y_train_chars for word in sentence)
        y_padded_chars = list()
        #for sentence in y_train_chars: 
            #y_padded_chars.append([word + [0] * (max_char_length - len(word)) for word in sentence])
            
        for i in range(len(y_padded_chars)): #0->253
            for j in range(len(y_padded_chars[i])):
                temp=list()
                for k in range(max_char_length - len(y_padded_chars[i][j])):
                    temp.append(0)
                #print(temp)
                if(len(temp)!=0):
                    y_padded_chars[i][j].extend(temp)
        self.tensor_y_chars = torch.tensor(y_padded_chars)

        print(len(self.tensor_x_words),len(self.tensor_x_words[0]))
        print(self.tensor_x_words[0], self.tensor_x_chars[0], self.tensor_y_chars[0])
        

    def __len__(self):
        return len(self.x_train_letters), len(self.x_train_words)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    def __getitem__(self, idx):
        return self.x_train_letters[idx], self.y_train_letters_numbered[idx], self.x_train_words[idx], self.y_train_words_numbered[idx]


<h4> Word2vec Feature Extraction </h4>

In [22]:
dataset = ArabicDataset(x_train_words_indices, x_train_chars, y_train_chars, one_hot_enconding)

278416 253 13 16
278416 253 13 23
253
278416 253 13 23
278416 253


IndexError: index 0 is out of bounds for dimension 0 with size 0

In [ ]:
print(dataset.tensor_x_words[0])

<h4> applying Bi-LSTM </h4>

In [ ]:
class Bi_Lstm(nn.Module):

  def __init__(self, embedding_dim: tuple, x_train_padded_size: int, hidden_size_layer_words: int, hidden_size_layer_chars: int, classes: int=14):
      
      self.embedding=nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim=embedding_dim[1], _weight=[embedding_matrix], input_length=x_train_padded_size)
      self.lstm_first_layer = nn.LSTM(input_size=(x_train_padded_size,embedding_dim[1]), hidden_size=(#noSentences,x_train_padded_size,embedding_dim[1]), num_layers=1, batch_first=True , bias=True, bidirectional=True)
      self.lstm_sec_layer = nn.LSTM(input_size=, hidden_size=hidden_size_layer2, num_layers=1, batch_first=True , bias=True, bidirectional=True)
      self.linear = nn.Softmax(hidden_size_layer2, classes, bias=True)

  def forward(self,sentences,dataset):
      feature_vector = self.embedding(sentences)
      lstm_output = self.lstm_first_layer(feature_vector)

      # looping over lstm_output to remove unwanted feature vectors
      no_of_sentences, length_of_sentence, feature_vector_size = lstm_output.shape

      lstm_output_without_padding = list()
      for i in range(no_of_sentences):
          #dy kda list of 1d dimension
          lstm_output_without_padding.extend(lstm_output[i, 1:len(dataset.x_train_words[i])-1, :])


      one_hot_enconding = np.eye(37)

      x_train_letter_concatenated = []

      for i in range(len(dataset.x_train_letters)):
          concatenated_arrays = []
          for letter in dataset.x_train_letters[i]:
              concatenated_arrays.append(np.concatenate([one_hot_enconding[dataset.arabic_alphabet[letter]], lstm_output_without_padding[i]]))
          x_train_letter_concatenated.extend(concatenated_arrays)

      final_output=self.lstm_sec_layer(x_train_letter_concatenated)


      #[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل']]
      return final_output


In [ ]:
# print(lstm_output[1][0:12])
# print(x_train_padded.shape)

## testing second model for char level encoding

In [ ]:
# looping over lstm_output to remove unwanted feature vectors
no_of_sentences, length_of_sentence, feature_vector_size = lstm_output.shape

lstm_output_without_padding = list()
for i in range(no_of_sentences):
    lstm_output_without_padding.extend(lstm_output[i, 1:len(dataset.x_train_words[i])-1, :])

# print(lstm_output_without_padding[0:6])


# x_train_letter_concatinated = [[np.concatenate(one_hot_enconding[letter_index], lstm_output_without_padding[i]) for letter_index in x_train_letter_indices[i]] for i in range(len(dataset.x_train_letters))]

x_train_letter_concatenated = []

for i in range(len(dataset.x_train_letters)):
    concatenated_arrays = []
    for letter in dataset.x_train_letters[i]:
        concatenated_arrays.append(np.concatenate([one_hot_enconding[dataset.arabic_alphabet[letter]], lstm_output_without_padding[i]]))
    x_train_letter_concatenated.extend(concatenated_arrays)


#[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل']]

In [ ]:
y_train_letter = [dataset.harakat[diacritic] for diacritic in dataset.y_train_letters]

## constructing model 2

In [ ]:
bi_lstm_model_char = Sequential()
first_bidirectional_layer = Bidirectional(LSTM(units=64, return_sequences=True))
bi_lstm_model_char.add(Dense(255, activation='softmax'))

bi_lstm_model_char.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bi_lstm_model_char.fit(np.array(x_train_letter_concatenated), np.array(y_train_letter), epochs=10, batch_size=32)
score = bi_lstm_model_char.evaluate(np.array(x_train_letter_concatenated), np.array(y_train_letter))
print("Test accuracy:", score[1])

bi_lstm_model_char.save("char-model")

In [ ]:
print(x_train_padded.shape)

In [ ]:
word_input = Input(shape=(max_word_length,))
word_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim)(word_input)
word_lstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(word_embedding)